# Exercise - Regression


The data set for this exercise includes information on house sales in King County, WA (between May 2014 and May 2015). (Each row in the data set pertains to one house. There is a total of 21,613 houses in the data set). Use this data set to predict the sale price of a house (i.e., the `price` column) based on the characteristics of the house. A model can be helpful for buyers, sellers, realtors, and lenders.

## Description of Variables

The description and type of each variable is provided in "KC house data - Data Dictionary.docx". Make sure to read this document to learn about the variables.

## Goal

Use the **kc_house_data.csv** data set and build a model to predict **price**. <br>

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

housing = pd.read_csv("kc_house_data.csv")
housing.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,432000,5.0,2.75,2060.0,329903.0,1.5,0,3,5,7.0,2060,0,1989.0,0,zip_98022,47.1776,-121.944,2240,220232.0
1,170000,2.0,1.00,810.0,8424.0,1.0,0,0,4,6.0,810,0,1959.0,0,zip_98023,47.3286,-122.346,820,8424.0
2,235000,3.0,1.00,960.0,5030.0,1.0,0,0,3,7.0,960,0,1955.0,0,zip_98118,47.5611,-122.280,1460,5400.0
3,350000,2.0,1.00,830.0,5100.0,1.0,0,0,4,7.0,830,0,1942.0,0,zip_98126,47.5259,-122.379,1220,5100.0
4,397380,2.0,1.00,1030.0,5072.0,1.0,0,0,3,6.0,1030,0,1924.0,1958,zip_98115,47.6962,-122.294,1220,6781.0


In [3]:
housing.shape

(19606, 19)

# Split data (train/test)

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(housing, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [6]:
train_target = train['price']
test_target = test['price']

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

##  Identify the numeric, binary, and categorical columns

In [7]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [8]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['waterfront']

In [9]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [10]:
binary_columns

['waterfront']

In [11]:
numeric_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [12]:
categorical_columns

['zipcode']

# Pipeline

In [13]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

In [17]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='drop')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [18]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<13724x87 sparse matrix of type '<class 'numpy.float64'>'
	with 233340 stored elements in Compressed Sparse Row format>

In [19]:
train_x.shape

(13724, 87)

# Tranform: transform() for TEST

In [20]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<5882x87 sparse matrix of type '<class 'numpy.float64'>'
	with 100008 stored elements in Compressed Sparse Row format>

In [21]:
test_x.shape

(5882, 87)

# Calculate the Baseline

In [23]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_target)

DummyRegressor()

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
# This is the baseline Train RMSE

dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_target, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 181098.44016311975


In [26]:
# This is the baseline Test RMSE

dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_target, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 181941.86906250828


# Train a SGD model (with no regularization)

In [28]:
from sklearn.linear_model import SGDRegressor 

# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.01) 

sgd_reg.fit(train_x, train_target)



SGDRegressor(max_iter=100, penalty=None)

In [29]:
# Create a new DataFrame

predictions = pd.DataFrame(sgd_reg.predict(test_x), columns=['Predicted'])

predictions

,Predicted
0,515897.768917
1,501480.154875
2,583232.205227
3,666229.980285
4,443441.587593
...,...
5877,244385.445865
5878,345872.525260
5879,459830.172574
5880,901146.530167


In [30]:
# Add the actual to the same DataFrame

predictions['Actual'] = np.array(test_target)

predictions

,Predicted,Actual
0,515897.768917,452000
1,501480.154875,396450
2,583232.205227,615000
3,666229.980285,563225
4,443441.587593,314963
...,...,...
5877,244385.445865,299000
5878,345872.525260,328000
5879,459830.172574,499900
5880,901146.530167,830005


### Generate the error metrics

In [31]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 77339.04234004956


In [32]:
#Test RMSE
reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 78870.15592322465


# Try L1 Regularization in SGD

In [44]:
#Stochastic Gradient:

sgd_reg_L2 = SGDRegressor(max_iter=500, penalty='l2', alpha = 0.1, eta0=0.01)

sgd_reg_L2.fit(train_x, train_target)

SGDRegressor(alpha=0.1, max_iter=500)

### Generate the error metrics

In [45]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 99659.81936899477


In [46]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 99763.29474494397


# Try L2 Regularization in SGD

In [48]:
#Stochastic Gradient:
sgd_reg_L1 = SGDRegressor(max_iter=500, penalty='l1', alpha = 0.1, eta0=0.01)

sgd_reg_L1.fit(train_x, train_target)

SGDRegressor(alpha=0.1, max_iter=500, penalty='l1')

### Generate the error metrics

In [50]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 77361.02467331389


In [52]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 78827.33048214982


# Try ElasticNet in SGD

In [55]:
#Stochastic Gradient:
sgd_reg_elastic = SGDRegressor(max_iter=500, penalty='elasticnet', l1_ratio=0.5, alpha = 0.1, 
                          eta0=0.01)
sgd_reg_elastic.fit(train_x, train_target)

SGDRegressor(alpha=0.1, l1_ratio=0.5, max_iter=500, penalty='elasticnet')

### Generate the error metrics

In [58]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 96151.06550499819


In [59]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 96433.82409146945


# Create Polynomial Features

Create polynomial features with degree = 2. 

In [60]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)

In [61]:
pol_lin_reg = SGDRegressor(max_iter=1000, penalty=None, eta0=0.01) 

pol_lin_reg.fit(train_x_poly, train_target)

SGDRegressor(penalty=None)

# Try L2 Regularization in SGD (with polynomial features)

In [62]:
#Stochastic Gradient:

sgd_reg_L2_pol = SGDRegressor(max_iter=50, penalty='l2', alpha = 0.1, eta0=0.01)

sgd_reg_L2_pol.fit(train_x_poly, train_target)

SGDRegressor(alpha=0.1, max_iter=50)

### Generate the error metrics